# Swallow

<https://huggingface.co/tokyotech-llm/Swallow-13b-instruct-hf>

Our Swallow model has undergone continuous pre-training from the Llama 2 family, primarily with the addition of Japanese language data. The tuned versions use supervised fine-tuning (SFT). Links to other models can be found in the index.

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

c:\Users\hamada\workspace\test\llm\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
# Proxy の無効化

!set HTTP_PROXY=""
!set HTTPS_PROXY=""

import sys,os,os.path

os.environ['HTTP_PROXY']=""
os.environ['HTTPS_PROXY']=""

In [16]:
# model_name = "tokyotech-llm/Swallow-70b-instruct-hf"
model_name = "tokyotech-llm/Swallow-13b-instruct-hf"
# model_name = "tokyotech-llm/Swallow-7b-instruct-hf"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16, low_cpu_mem_usage=True, device_map="auto")

model-00001-of-00006.safetensors: 100%|██████████| 4.99G/4.99G [01:30<00:00, 54.9MB/s]
model-00002-of-00006.safetensors: 100%|██████████| 4.97G/4.97G [01:25<00:00, 58.4MB/s]
model-00003-of-00006.safetensors: 100%|██████████| 4.93G/4.93G [03:52<00:00, 21.2MB/s]
model-00004-of-00006.safetensors: 100%|██████████| 4.93G/4.93G [02:38<00:00, 31.1MB/s]
model-00005-of-00006.safetensors: 100%|██████████| 4.93G/4.93G [01:20<00:00, 61.2MB/s]
model-00006-of-00006.safetensors: 100%|██████████| 1.50G/1.50G [00:48<00:00, 31.0MB/s]
generation_config.json: 100%|██████████| 203/203 [00:00<?, ?B/s] 


In [17]:
PROMPT_DICT = {
    "prompt_input": (
        "以下に、あるタスクを説明する指示があり、それに付随する入力が更なる文脈を提供しています。"
        "リクエストを適切に完了するための回答を記述してください。\n\n"
        "### 指示:\n{instruction}\n\n### 入力:\n{input}\n\n### 応答:"
    ),
    "prompt_no_input": (
        "以下に、あるタスクを説明する指示があります。"
        "リクエストを適切に完了するための回答を記述してください。\n\n"
        "### 指示:\n{instruction}\n\n### 応答:"
    ),
}


def create_prompt(instruction, input=None):
    """
    Generates a prompt based on the given instruction and an optional input.
    If input is provided, it uses the 'prompt_input' template from PROMPT_DICT.
    If no input is provided, it uses the 'prompt_no_input' template.

    Args:
        instruction (str): The instruction describing the task.
        input (str, optional): Additional input providing context for the task. Default is None.

    Returns:
        str: The generated prompt.
    """
    if input:
        # Use the 'prompt_input' template when additional input is provided
        return PROMPT_DICT["prompt_input"].format(instruction=instruction, input=input)
    else:
        # Use the 'prompt_no_input' template when no additional input is provided
        return PROMPT_DICT["prompt_no_input"].format(instruction=instruction)

In [26]:
# Example usage
instruction = "入力はとあるソフトウェアのインターフェースを説明した文章です。入力内容を英語に翻訳してください。"
input_example = "View のコンストラクタが引数として ViewModel を要求する場合は、このメソッド引数の ViewModel を渡します。"
prompt = create_prompt(instruction, input_example)

input_ids = tokenizer.encode(
    prompt,
    add_special_tokens=False,
    return_tensors="pt"
)

tokens = model.generate(
    input_ids.to(device=model.device),
    max_new_tokens=128,
    temperature=0.99,
    top_p=0.95,
    do_sample=True,
)

out = tokenizer.decode(tokens[0], skip_special_tokens=True)
print(out)

以下に、あるタスクを説明する指示があり、それに付随する入力が更なる文脈を提供しています。リクエストを適切に完了するための回答を記述してください。

### 指示:
入力はとあるソフトウェアのインターフェースを説明した文章です。入力内容を英語に翻訳してください。

### 入力:
View のコンストラクタが引数として ViewModel を要求する場合は、このメソッド引数の ViewModel を渡します。

### 応答:If the constructor of the View requires ViewModel as an argument, pass the ViewModel argument passed to the constructor into the method argument.


In [22]:
# read input file
file_name = "../inputs.txt"
with open(file_name, "r", encoding="utf-8") as f:
    lines = f.readlines()

for line in lines:
    prompt = create_prompt(instruction, line)

    input_ids = tokenizer.encode(
        prompt,
        add_special_tokens=False,
        return_tensors="pt"
    )

    tokens = model.generate(
        input_ids.to(device=model.device),
        max_new_tokens=128,
        temperature=0.99,
        top_p=0.95,
        do_sample=True,
    )

    out = tokenizer.decode(tokens[0], skip_special_tokens=True)
    print(out)
    print("/n===/n")

以下に、あるタスクを説明する指示があり、それに付随する入力が更なる文脈を提供しています。リクエストを適切に完了するための回答を記述してください。

### 指示:
入力内用はとあるソフトウェアのインターフェースを説明した文章です。入力内用を英語に翻訳してください。また、応答には翻訳した文章のみを出力してください

### 入力:
指定された ID に紐づけられた型を取得する


### 応答:Get the type associated with the specified ID.
以下に、あるタスクを説明する指示があり、それに付随する入力が更なる文脈を提供しています。リクエストを適切に完了するための回答を記述してください。

### 指示:
入力内用はとあるソフトウェアのインターフェースを説明した文章です。入力内用を英語に翻訳してください。また、応答には翻訳した文章のみを出力してください

### 入力:
View のコンストラクタが引数として ViewModel を要求する場合は、このメソッド引数の ViewModel を渡します。


### 応答:
View のコンストラクタが引数として ViewModel を要求する場合は、このメソッド引数の ViewModel を渡します。

以下に、あるタスクを説明する指示があり、それに付随する入力が更なる文脈を提供しています。リクエストを適切に完了するための回答を記述してください。

### 指示:
入力内用はとあるソフトウェアのインターフェースを説明した文章です。入力内用を英語に翻訳してください。また、応答には翻訳した文章のみを出力してください

### 入力:
カメラの動作状態を格納するクラスです。


### 応答:カメラの動作状態を格納するクラスです。
以下に、あるタスクを説明する指示があり、それに付随する入力が更なる文脈を提供しています。リクエストを適切に完了するための回答を記述してください。

### 指示:
入力内用はとあるソフトウェアのインターフェースを説明した文章です。入力内用を英語に翻訳してください。また、応答には翻訳した文章のみを出力してください

### 入力:
IN/OUT状態を表す列挙型です。


### 応答:状態を表す列挙型IN/OUTです。
以下に、あるタスクを説明する指示があり、